In [ ]:
from __future__ import print_function

import numpy as np

from sklearn import __version__ as sklearn_version
print('Sklearn version:', sklearn_version)

# The data

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.


In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',
                 remove=('headers', 'footers', 'quotes'),
                 categories=categories, shuffle=True, random_state=42)


## Build a pipeline

In [ ]:
#Define the pipeline

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVect...),
                     ('tfidf', TfidfTran...),
# ---------------------------------------------
                     ('clf', ...),
# ---------------------------------------------
                    ])

# Fit all the pipeline
text_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
#Evaluate test data
twenty_test = fetch_20newsgroups(subset='test',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories, 
                    shuffle=True, random_state=42)

predicted = text_clf.predict(twenty_test.data)

print('Test accuracy:', np.mean(predicted == twenty_test.target))

## Change classifier in the pipeline
    - LinearSVC
    - k-NN
    - Random forest

In [ ]:
from sklearn.svm import LinearSVC
text_clf_svm = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
# ---------------------------------------------
                     ('clf', ...),
# ---------------------------------------------
                    ])

#Fit
_ = text_clf_svm.fit(twenty_train.data, twenty_train.target)

# Predict
predicted = text_clf_svm.predict(twenty_test.data)

# Evaluate accuracy
print('Test accuracy:', np.mean(predicted == twenty_test.target))        

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
# ---------------------------------------------
                     ('clf', ...),
# ---------------------------------------------
                    ])

_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')),
                     ('tfidf', TfidfTransformer()),
# ---------------------------------------------
                     ('clf', ...),
# ---------------------------------------------
                    ])

_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

## Use features from a factorization instead the provided by the tf-idf

In [ ]:
# Text preprocessing, tokenizing and filtering of stopwords
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=5000,
                                stop_words='english')
X_train_counts = tf_vectorizer.fit_transform(twenty_train.data)
X_train_counts.shape

In [ ]:
from sklearn.decomposition import  LatentDirichletAllocation

n_components = 6
n_top_words = 20

lda = LatentDirichletAllocation(n_components=n_components,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(X_train_counts)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

## Pipeline with factorization

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# ---------------------------------------------
text_lda_knn = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=10000, stop_words='english')),
                         ('lda', ...),
                         ('clf', ...)
                        ])
# ---------------------------------------------

                         
_ = text_lda_knn.fit(twenty_train.data, twenty_train.target)
predicted = text_lda_knn.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# ---------------------------------------------
text_lda_rf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=2, max_features=10000, stop_words='english')),
                         ('lda', ...),
                         ('clf', ...),
                        ])
# ---------------------------------------------

                         
_ = text_lda_rf.fit(twenty_train.data, twenty_train.target)

predicted = text_lda_rf.predict(twenty_test.data)
print('Test accuracy:', np.mean(predicted == twenty_test.target))

## Optimize a pipeline

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define estimator. No parameters of the search
# ---------------------------------------------
clf = Pipeline([('vect', ...),
                ('tfidf', ...), # tf-idf
                ('clf', ...), # LinearSVC
                ])
# ---------------------------------------------

# Specify parameters and distributions to sample from
# Parameters of pipelines can be set using ‘__’ separated parameter names:
# ---------------------------------------------
param_dist = {"vect__max_features": ..., 
              "vect__stop_words": ..., 
              "clf__C": ...}
# ---------------------------------------------

# Define randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search)

# Run the randomized search
random_search.fit(twenty_train.data, twenty_train.target)

print("Done!")

In [ ]:
# Load dictionary of search results to a Pandas dataframe
import pandas as pd

df_cv_results = pd.DataFrame.from_dict(random_search.cv_results_)
df_cv_results

In [ ]:
print('Best params:', random_search....)

In [ ]:
# Score & evaluate test data using the best estimator

# ---------------------------------------------
predicted = random_search...(twenty_test.data)
# ---------------------------------------------

print('Test accuracy:', np.mean(predicted == twenty_test.target))

## Aditional metrics for multiclass classification

In [ ]:
from sklearn import metrics

print(metrics.classification_report(twenty_test.target, 
                                    predicted,
                                    target_names=twenty_test.target_names))

In [ ]:
metrics.confusion_matrix(twenty_test.target, predicted)